In [ ]:
import warnings
warnings.filterwarnings("ignore")

import climt
import gfs_dynamical_core as gfs
from sympl import (TimeDifferencingWrapper, PlotFunctionMonitor)
import numpy as np
from datetime import timedelta
import time
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import matplotlib.transforms as mtransforms
import xarray as xr
import os
import pickle
import gzip

model_time_step = timedelta(minutes=20)

# Create components
convection = climt.EmanuelConvection()
boundary= TimeDifferencingWrapper(climt.SimplePhysics())
radiation = climt.GrayLongwaveRadiation()
slab_surface = climt.SlabSurface()
optical_depth = climt.Frierson06LongwaveOpticalDepth(linear_optical_depth_parameter=1, longwave_optical_depth_at_equator=6)

dycore = gfs.GFSDynamicalCore(
    [boundary, radiation, convection, slab_surface], number_of_damped_levels=5
)

grid = climt.get_grid(nx=128, ny=64)

# Create model state
my_state = climt.get_default_state([dycore], grid_state=grid)

# Set initial/boundary conditions
latitudes = my_state['latitude'].values
longitudes = my_state['longitude'].values

surface_shape = latitudes.shape

# Set initial/boundary conditions
sw_max = 150
sw_min = 0

sw_flux_profile = (sw_max*(1+1.4*(1/4*(1-3*np.sin(np.radians(latitudes-10))**2))))
sw_flux_profile[np.where(latitudes<-80)]=sw_max*(1+1.4/4*(1-3))

my_state['downwelling_shortwave_flux_in_air'].values[:] = sw_flux_profile[np.newaxis, :]
my_state.update(optical_depth(my_state))
my_state['longwave_optical_depth_on_interface_levels'][0]=0

my_state['surface_temperature'].values[:] = 290.
my_state['ocean_mixed_layer_thickness'].values[:] = 2
my_state['soil_layer_thickness'].values[:] = 1

my_state['eastward_wind'].values[:] = np.random.randn(
    *my_state['eastward_wind'].shape)


# Spinup model for a year
for i in range(1):#26280 one year

    print(my_state.keys())

    with gzip.open('model_sigma', 'wb') as f:
        sig=[my_state['atmosphere_hybrid_sigma_pressure_a_coordinate_on_interface_levels'],
                    my_state['atmosphere_hybrid_sigma_pressure_b_coordinate_on_interface_levels']]
        pickle.dump(sig, f)

    diag, my_state = dycore(my_state, model_time_step)
    my_state.update(diag)
    my_state['time'] += model_time_step

dict_keys(['atmosphere_hybrid_sigma_pressure_a_coordinate_on_interface_levels', 'atmosphere_hybrid_sigma_pressure_b_coordinate_on_interface_levels', 'surface_air_pressure', 'time', 'air_pressure', 'air_pressure_on_interface_levels', 'longitude', 'latitude', 'height_on_ice_interface_levels', 'air_temperature', 'eastward_wind', 'northward_wind', 'divergence_of_wind', 'atmosphere_relative_vorticity', 'surface_geopotential', 'surface_temperature', 'specific_humidity', 'surface_specific_humidity', 'longwave_optical_depth_on_interface_levels', 'cloud_base_mass_flux', 'downwelling_longwave_flux_in_air', 'downwelling_shortwave_flux_in_air', 'upwelling_longwave_flux_in_air', 'upwelling_shortwave_flux_in_air', 'surface_upward_latent_heat_flux', 'surface_upward_sensible_heat_flux', 'surface_thermal_capacity', 'surface_material_density', 'upward_heat_flux_at_ground_level_in_soil', 'heat_flux_into_sea_water_due_to_sea_ice', 'area_type', 'soil_layer_thickness', 'ocean_mixed_layer_thickness', 'heat_c